In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
from PortfolioToolboxQP.PortfolioToolboxQP.Qp_Solver import qp_solver_cpp as solve

Il faut remplir le code suivant avec ce qu'on a calculé dans les autres fichiers. Faut juste faire attention à ce qu'on est les mêmes colonnes pour tous sinon on va avoir des problèmes de dimensions.

In [3]:
sigma = np.load("data/cov_matrix_sigma.npy")
print(sigma.shape)
CI = pd.read_csv("data/CarbonIntensity.csv")
print(CI.shape)
CI0 = CI["CI_Scope12_FY23"].values.reshape(-1, 1)
print(type(CI0))
print(type(sigma))

bench = (CI["Weight"].values / CI["Weight"].sum()).reshape(-1, 1)

print(bench.shape)
print(CI0.shape)
R = 0.35  # reduction_rate cf net_zero_emissions for other values

(1131, 1131)
(1131, 50)
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
(1131, 1)
(1131, 1)


J'ai pris le problème le plus simple avec le moins de contrainte (min 1/2(x-b)Tsigma(x-b) avec comme contraintes la carbone intensity, et le bail C0 des long only constraints [x entre 0 et 1 et qui se somme à 1])

In [4]:
y = solve(
    Q=sigma,
    p=None,
    G=np.diag(CI0[:,0]),
    h=(1 - R) * CI0*bench - CI0*bench,
    A= np.ones((bench.shape[0],1)).T,
    b=-np.ones(bench.shape).T @ bench + 1,
    lb= - bench,
    ub=np.ones(bench.shape) - bench,
)

""" x is what we want : here the relative weights on the differents firms. The following Y is what we call "x-b". 
 We have to add the benchmark at the end

                            minimize 0.5*Y.T@Q@Y + p@Y under constraints AY = B and GY <= h and  lb <= Y <= ub
    
    all parameters are ndarray

    Q: sigma for us
    p: 0

    CI*(X=Y+b) <= (1-R)CI_0*b donc CI*Y <= (1-R)*CI_0*b - CI*b
    G: Matrix of inequalty constraints → np.diag(CI0[:,0])
    h: Vector of inequality constraints → (1 - R) * CI0*bench - CI0*bench
    1.T(Y+b)=1 donc 1TY = - 1Tb + 1
    A: Matrix of equality constraints → 1T
    b: Vector of equality constraints → -1Tb + 1
    
    lb: Lower bound for Y : for x : 0
    ub: Upper bound for Y : for x : 1

    return: optimal vector if found : Y=X-b
    """

c:\Users\vahye\Desktop\ENSAE\projet_finance\net-zero-portfolio\PortfolioToolboxQP\PortfolioToolboxQP\Qp_Solver.py:137: UserWarning: Warning : The p matrix in the QP problem was set to 0 as it was either empty or None -- Continuing optimization !
  warnings.warn(
c:\Users\vahye\Desktop\ENSAE\projet_finance\net-zero-portfolio\PortfolioToolboxQP\PortfolioToolboxQP\Qp_Solver.py:189: UserWarning: Warning : b was reshaped as its shape was not of form (n,)
  warnings.warn("Warning : b was reshaped as its shape was not of form (n,)")
c:\Users\vahye\Desktop\ENSAE\projet_finance\net-zero-portfolio\PortfolioToolboxQP\PortfolioToolboxQP\Qp_Solver.py:202: UserWarning: Warning : h was reshaped as its shape was not of form (n,)
  warnings.warn("Warning : h was reshaped as its shape was not of form (n,)")


' x is what we want : here the relative weights on the differents firms. The following Y is what we call "x-b". \n We have to add the benchmark at the end\n\n                            minimize 0.5*Y.T@Q@Y + p@Y under constraints AY = B and GY <= h and  lb <= Y <= ub\n    \n    all parameters are ndarray\n\n    Q: sigma for us\n    p: 0\n\n    CI*(X=Y+b) <= (1-R)CI_0*b donc CI*Y <= (1-R)*CI_0*b - CI*b\n    G: Matrix of inequalty constraints → np.diag(CI0[:,0])\n    h: Vector of inequality constraints → (1 - R) * CI0*bench - CI0*bench\n    1.T(Y+b)=1 donc 1TY = - 1Tb + 1\n    A: Matrix of equality constraints → 1T\n    b: Vector of equality constraints → -1Tb + 1\n    \n    lb: Lower bound for Y : for x : 0\n    ub: Upper bound for Y : for x : 1\n\n    return: optimal vector if found : Y=X-b\n    '

In [5]:
y = y[:,np.newaxis]
x = y + bench
print(x)
print(x.shape)
print(x.sum())
tracking_error = 0.5 * y.T @ sigma @ y
print(tracking_error)

[[0.00067043]
 [0.00018278]
 [0.00012406]
 ...
 [0.0097468 ]
 [0.00039805]
 [0.00044735]]
(1131, 1)
0.9999999999999999
[[2.07044389e-06]]


In [6]:
x_list = []
te = []
R_ = [0.35,0.35,0.35,0.35]

for year in range(3):
    R = R_[year]
    ye = 23+year
    column = "CI_Scope12_FY"+str(ye)
    print(column)
    CI_year = CI[column].values.reshape(-1, 1)
    y = solve(
        Q = sigma,
        p = None,
        G = np.diag(CI_year[:,0]),
        h = (1 - R) * CI0 * bench - CI_year * bench,
        A = np.ones((bench.shape[0],1)).T,
        b =-np.ones(bench.shape).T @ bench + 1,
        lb = - bench,
        ub =np.ones(bench.shape) - bench,
    )
    y = y[:,np.newaxis]
    x = y + bench
    x_list.append(x)
    print(x.sum())
    tracking_error = 0.5 * y.T @ sigma @ y
    te.append(tracking_error)

    

CI_Scope12_FY23


c:\Users\vahye\Desktop\ENSAE\projet_finance\net-zero-portfolio\PortfolioToolboxQP\PortfolioToolboxQP\Qp_Solver.py:137: UserWarning: Warning : The p matrix in the QP problem was set to 0 as it was either empty or None -- Continuing optimization !
  warnings.warn(
c:\Users\vahye\Desktop\ENSAE\projet_finance\net-zero-portfolio\PortfolioToolboxQP\PortfolioToolboxQP\Qp_Solver.py:189: UserWarning: Warning : b was reshaped as its shape was not of form (n,)
  warnings.warn("Warning : b was reshaped as its shape was not of form (n,)")
c:\Users\vahye\Desktop\ENSAE\projet_finance\net-zero-portfolio\PortfolioToolboxQP\PortfolioToolboxQP\Qp_Solver.py:202: UserWarning: Warning : h was reshaped as its shape was not of form (n,)
  warnings.warn("Warning : h was reshaped as its shape was not of form (n,)")


0.9999999999999999
CI_Scope12_FY24
Valeur d'échec : 2


ValueError: Accuracy insufficient to satisfy convergence criterion